# Libraries Loading

In [1]:
import numpy as np
import pandas as pd
import arabic
import pyarabic
import helpers
from helpers import Clean_data,separate_token_with_dicrites,save_h5
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import array, argmax
import numpy as np
import os,errno,re,sys
import pickle
from itertools import product 
from pyarabic.araby import strip_tashkeel, strip_tatweel
import h5py
import random as rn
##########################
rn.seed(123456)
np.random.seed(123456)
input_data_path = "../data/Almoso3a_Alshe3rya/data/All_ksaied.csv"

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
try:
    os.makedirs("../data/Encoded/8bits/WithoutTashkeel/Eliminated/")
    os.makedirs("../data/Encoded/8bits/WithoutTashkeel/Full_Data/")
    os.makedirs("../data/Encoded/8bits/WithTashkeel/Eliminated/")
    os.makedirs("../data/Encoded/8bits/WithTashkeel/Full_Data/")
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise
print("Paths created")

Paths created


# Data Loading

In [3]:
input_data = pd.read_csv(input_data_path, index_col=0)
input_data.info(memory_usage='deep')
bahr = 'Bahr'
bayt = 'Bayt_Text'
#input_data  = input_data[[bayt, bahr]]
bahr = 'البحر'
bayt = 'البيت'
input_data  = input_data[[bayt, bahr]]
input_data.columns = ['Bayt_Text','Bahr']
our_alphabets = "".join(arabic.alphabet) + "".join(arabic.tashkeel)+" "
our_alphabets = "".join(our_alphabets)
input_data['Bayt_Text'] = input_data['Bayt_Text'].apply(lambda x: re.sub(r'[^'+our_alphabets+']','',str(x))).apply(lambda x: re.sub(r'  *'," ",x)).apply(lambda x: re.sub(r'ّ+', 'ّ', x)).apply(lambda x: x.strip())
input_data.shape

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1831770 entries, 0 to 1835252
Data columns (total 8 columns):
العصر           object
الشاعر          object
الديوان         object
القافية         object
البحر           object
الشطر الايسر    object
الشطر الايمن    object
البيت           object
dtypes: object(8)
memory usage: 1.8 GB


(1831770, 2)

# Data Analysis

In [4]:
input_data.iloc[:, 1].value_counts()


الطويل          405378
الكامل          363486
البسيط          239974
الخفيف          156049
الوافر          140560
الرجز           117572
&nbsp;           92334
الرمل            79015
المتقارب         63818
السريع           58084
موشح             30060
المنسرح          28357
المجتث           17884
المديد            7829
الهزج             7541
عامي              7074
المتدارك          5037
شعر التفعيلة      3232
الدوبيت           2713
شعر حر            2135
المواليا          1644
السلسلة            907
المقتضب            799
المضارع            288
Name: Bahr, dtype: int64

# Data Filteration

In [5]:
elminated_classic_bohor = ['الوافر', 'المنسرح','المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط']

elminated_classic_bohor

['الوافر',
 'المنسرح',
 'المجتث',
 'المتقارب',
 'الكامل',
 'الطويل',
 'السريع',
 'الرمل',
 'الرجز',
 'الخفيف',
 'البسيط']

In [6]:
full_bohor_classes = ['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']
full_bohor_classes

['الوافر',
 'المنسرح',
 'المديد',
 'المجتث',
 'المتقارب',
 'الكامل',
 'الطويل',
 'السريع',
 'الرمل',
 'الرجز',
 'الخفيف',
 'البسيط',
 'المقتضب',
 'الهزج',
 'المضارع',
 'المتدارك']

## Data Filter

In [7]:
full_filtered_data = input_data.loc[input_data['Bahr'].isin(full_bohor_classes)]
full_filtered_data['Bayt_Text'] = full_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
full_filtered_data['Bayt_Text'] = full_filtered_data['Bayt_Text'].apply(helpers.factor_shadda_tanwin)
full_filtered_data.head()

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Bayt_Text,Bahr
0,خَليلَيْيَ لا تَستَعجِلا أَن تَزَوْوَدا وَأَن ...,الطويل
1,فَما لَبَثُنْ يَومنْ بِسابِقِنْ مَغنَمِنْ وَلا...,الطويل
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَتَنْ وَتَس...,الطويل
3,لَعَمرُكَ ما نَفسُنْ بِجِدِنْ رَشيدَةتِنْ تُؤا...,الطويل
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمْمَتُنْ وَأَفر...,الطويل


In [ ]:
eliminated_filtered_data = input_data.loc[input_data['Bahr'].isin(elminated_classic_bohor)]
eliminated_filtered_data['Bayt_Text'] = eliminated_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
eliminated_filtered_data['Bayt_Text'] = eliminated_filtered_data['Bayt_Text'].apply(helpers.factor_shadda_tanwin)
eliminated_filtered_data.head()

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Bayt_Text,Bahr
0,خَليلَيْيَ لا تَستَعجِلا أَن تَزَوْوَدا وَأَن ...,الطويل
1,فَما لَبَثُنْ يَومنْ بِسابِقِنْ مَغنَمِنْ وَلا...,الطويل
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَتَنْ وَتَس...,الطويل
3,لَعَمرُكَ ما نَفسُنْ بِجِدِنْ رَشيدَةتِنْ تُؤا...,الطويل
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمْمَتُنْ وَأَفر...,الطويل


# Encode Classes

In [11]:
full_data_label_encoder = LabelEncoder()
full_data_integer_encoded = full_data_label_encoder.fit_transform(list(full_filtered_data["Bahr"]))
# binary encode
full_data_onehot_encoder = OneHotEncoder(sparse=False)
full_data_integer_encoded = full_data_integer_encoded.reshape(len(full_data_integer_encoded), 1)
full_data_bohor_encoded = full_data_onehot_encoder.fit_transform(full_data_integer_encoded)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
save_h5('../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../encoders_full_dat.pickle", 'wb') as pre_file:
    pickle.dump(full_data_label_encoder, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ....')

saving preprocessors....
saved ....


In [12]:
### testing
#print('loading saved encoders from %s'%(encoders_file_name))
with open("../encoders_full_dat.pickle", 'rb') as f:
    encoders = pickle.load(f)
##testing 
inverted = encoders.inverse_transform([argmax(full_filtered_data["Bahr"][1,])])
inverted

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['البسيط'], dtype='<U8')

In [13]:
eliminated_data_label_encoder = LabelEncoder()
eliminated_data_integer_encoded = eliminated_data_label_encoder.fit_transform(list(eliminated_filtered_data["Bahr"]))
# binary encode
eliminated_data_onehot_encoder = OneHotEncoder(sparse=False)
eliminated_data_integer_encoded = eliminated_data_integer_encoded.reshape(len(eliminated_data_integer_encoded), 1)
eliminated_data_bohor_encoded = eliminated_data_onehot_encoder.fit_transform(eliminated_data_integer_encoded)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
save_h5('../data/Encoded/8bits/WithTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../encoders_eliminated_data.pickle", 'wb') as pre_file:
    pickle.dump(eliminated_data_bohor_encoded, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ...')

saving preprocessors....
saved ...


# Clean The Data

## WITH TASHKEEL

### FULL DATA

In [ ]:
bayt_len_full_withtashkeel = np.max((full_filtered_data.Bayt_Text.apply(separate_token_with_dicrites).apply(len)))
print("bayt_len_full_withtashkeel done..")
full_filtered_data_with_tashkeel_cleaned = Clean_data(data_frame = full_filtered_data,max_bayt_len = bayt_len_full_withtashkeel,verse_column_name = 'Bayt_Text')
print("full_cleaned_data_with_tashkeel cleaned ..")
full_cleaned_data_with_tashkeel_encoded = full_filtered_data_with_tashkeel_cleaned.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_full_withtashkeel))
print("full_cleaned_data_with_tashkeel_encoded encoded ..")
full_cleaned_data_with_tashkeel_encoded_stacked = np.stack(full_cleaned_data_with_tashkeel_encoded,axis=0)
save('../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', full_cleaned_data_with_tashkeel_encoded_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(full_cleaned_data_with_tashkeel_encoded_stacked.shape)

#### ELIMINATED DATA

In [22]:
bayt_len_eliminated_withtashkeel = np.max((eliminated_filtered_data.Bayt_Text.apply(separate_token_with_dicrites).apply(len)))
print("bayt_len_eliminated_withtashkeel done..")
eliminated_filtered_data_with_tashkeel_cleaned = Clean_data(data_frame = eliminated_filtered_data,max_bayt_len = bayt_len_eliminated_withtashkeel  ,verse_column_name = 'Bayt_Text')
print("eliminated_cleaned_data_with_tashkeel cleaned ..")
eliminated_cleaned_data_with_tashkeel_encoded = eliminated_filtered_data_with_tashkeel_cleaned.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_eliminated_withtashkeel))
print("eliminated_cleaned_data_with_tashkeel_encoded encoded ..")
eliminated_cleaned_data_with_tashkeel_encoded_stacked = np.stack(eliminated_cleaned_data_with_tashkeel_encoded,axis=0)
save_h5('../data/Encoded/8bits/WithTashkeel/Eliminated/eliminated_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', eliminated_cleaned_data_with_tashkeel_encoded_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(eliminated_cleaned_data_with_tashkeel_encoded_stacked.shape)
#encoded_byot = data_with_Tashkeel_cleaned.apply(lambda x: helpers.string_with_tashkeel_vectorizer_OneHot(x, maximum_bayt_len))

full_cleaned_data_with_tashkeel_encoded saved ..
(1670177, 84, 8)


## WITHOUT TASHKEEL

### FULL DATA

In [ ]:
full_filtered_data_without_tashkeel = full_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tashkeel)
bayt_len_full_without_tashkeel = np.max((full_filtered_data_without_tashkeel.apply(len)))
print("bayt_len_full_without_tashkeel done..")
full_cleaned_data_without_tashkeel_encoded = full_filtered_data_without_tashkeel.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_full_without_tashkeel))
print("full_filtered_data_without_tashkeel encoded ..")
full_filtered_data_without_tashkeel_staked = np.stack(full_cleaned_data_without_tashkeel_encoded,axis=0)
save('../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', full_filtered_data_without_tashkeel_staked) 
print("full_filtered_data_without_tashkeel_staked saved ..")
print(full_cleaned_data_without_tashkeel_encoded.shape)


bayt_len_full_without_tashkeel done..


### ELIMINATE

In [ ]:
eliminated_filtered_data_without_tashkeel = eliminated_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tashkeel)
bayt_len_eliminated_without_tashkeel = np.max((eliminated_filtered_data_without_tashkeel.apply(len)))
print("bayt_len_eliminated_without_tashkeel done..")
eliminated_cleaned_data_without_tashkeel_encoded = eliminated_filtered_data_without_tashkeel.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, bayt_len_eliminated_without_tashkeel))
print("eliminated_filtered_data_without_tashkeel encoded ..")
eliminated_filtered_data_without_tashkeel_staked = np.stack(eliminated_cleaned_data_without_tashkeel_encoded,axis=0)
save('../data/Encoded/8bits/WithoutTashkeel/Eliminated/eliminated_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', full_filtered_data_without_tashkeel_staked) 
print("eliminated_filtered_data_without_tashkeel_staked saved ..")
print(eliminated_cleaned_data_without_tashkeel_encoded.shape)


## Congratualtions :) Finished :) :) :)